In [7]:
# Load libraries
import numpy as np
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.losses import MeanAbsoluteError
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.optimizers import Adam


In [8]:
data = pd.read_csv('BS_data.csv')
data

,Company,Time,Total Assets,Total Liabilities,Total Equity,Total Shares Out. on Filing Date,Book Value / Share,Tangible Book Value,Tangible Book Value Per Share,Total Debt,Net Debt,Price
0,RHI,2013-12-01,1490.3,570.6,919.6,135.3,6.80,718.3,5.31,1.4,-274.3,41.99
1,RCL,2011-12-01,19804.4,11396.6,8407.8,217.5,38.74,7442.4,34.29,8507.2,8245.0,24.77
2,NVDA,2020-01-01,17315.0,5111.0,12204.0,612.0,19.93,11537.0,18.84,2643.0,-8254.0,236.43
3,STC,2012-12-01,1291.2,710.8,580.4,21.1,29.31,263.5,13.58,71.2,-137.4,26.00
4,XRAY,2010-12-01,3258.0,1348.0,1909.9,142.1,12.98,457.6,3.23,786.6,246.6,34.17
5,GWW,2016-12-01,5694.3,3788.5,1905.8,58.8,30.57,684.7,11.64,2247.1,1972.9,232.25
6,SKM,2015-12-01,24288.4,11223.5,13064.9,70.6,183.55,9386.2,132.93,7039.9,5790.9,20.15
7,BAX,2011-12-01,19073.0,12245.0,6828.0,560.3,11.74,3442.0,6.14,5195.0,2290.0,26.88
8,MAR,2012-12-01,6342.0,7627.0,-1285.0,312.3,-4.13,-3274.0,-10.53,2935.0,2847.0,37.27
9,SJM,2019-04-01,16711.3,8740.8,7970.5,113.7,70.08,-5059.2,-44.48,5959.9,5858.6,122.63


In [9]:
prices = data['Price'] 
data = data.drop(['Company', 'Time', 'Price'], axis = 1)

col_list = []
for col in data.columns:
    col_list.append(col)
    
for col in col_list:
    print(col)
    data[col] = data[col].astype(float) # Converting columns to floats
    print("Done")

# Normalising the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(data) 
data.loc[:,:] = scaled_values

data.insert(data.shape[1], 'Price', prices)

# Converting Data to Numpy Arrays
NpMatrix = data.to_numpy(dtype = None, copy = False)
X = NpMatrix[:,0:9] # Parameters
y = NpMatrix[:,9] # Price
print("X:", X)
print("\n")
print("Type X:", type(X))
print("Length of Individual X_train Vector:", len(X[1]))
print("Total Number of Training instances:", len(X))
print("\n")
print(y)
print("\n")
print("Type y:", type(y))
print("Length of Individual y_train vector", y[0])
print("Total number of y values", len(y))

Total Assets
Done
Total Liabilities
Done
Total Equity
Done
Total Shares Out. on Filing Date
Done
Book Value / Share
Done
Tangible Book Value
Done
Tangible Book Value Per Share
Done
Total Debt
Done
Net Debt
Done
X: [[1.63843386e-03 6.82611560e-04 2.81693214e-02 ... 1.34060792e-03
  2.92521072e-06 1.85568632e-01]
 [2.19349988e-02 1.36816489e-02 4.51864494e-02 ... 1.49174313e-03
  1.77752519e-02 2.00645972e-01]
 [1.91761262e-02 6.13437854e-03 5.38134120e-02 ... 1.41116898e-03
  5.52237995e-03 1.71446269e-01]
 ...
 [6.79067888e-03 4.97604015e-03 3.06086506e-02 ... 1.34050362e-03
  4.01380699e-03 1.89340224e-01]
 [2.35835177e-03 1.36510305e-03 2.83540777e-02 ... 1.34947368e-03
  1.52340795e-03 1.86742708e-01]
 [7.21358632e-04 8.08087212e-04 2.60515545e-02 ... 1.31145520e-03
  1.21500717e-03 1.86722887e-01]]


Type X: <class 'numpy.ndarray'>
Length of Individual X_train Vector: 9
Total Number of Training instances: 5000


[ 41.99  24.77 236.43 ...  49.01   8.07  34.69]


Type y: <class 'nump

In [10]:
number_of_features = len(X[1])

#Loss functions
# https://keras.io/api/losses/regression_losses/#meanabsoluteerror-class
# https://keras.io/api/losses/

# Create function returning a compiled network
def create_network():
    
    # Start neural network
    network = models.Sequential()

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=81, activation='relu', input_shape=(number_of_features,)))
    
    network.add(tf.keras.layers.Dropout(0.2))

    # Add fully connected layer with a sigmoid activation function
    network.add(layers.Dense(units=1, activation='relu'))

    # Compile neural network
    network.compile(#loss='mse', # Mean Squared Error
                    loss = MeanAbsoluteError(),
                    optimizer='adam', # Root Mean Square Propagation
                    metrics=['mae', 'mse']) # Accuracy performance metric
    
    # Return compiled network
    return network

In [11]:
neural_network = KerasRegressor(build_fn=create_network, 
                                 epochs= 750, 
                                 batch_size= 6)

In [12]:
# Evaluate neural network using three-fold cross-validation
scores = cross_val_score(neural_network, X, y, cv=5)

Train on 4000 samples
Epoch 1/750
4000/4000 [==============================] - 1s 303us/sample - loss: 60.3497 - mae: 60.3497 - mse: 20588.0020s - loss: 66.7939 - mae: 66.7939 - mse: 247
Epoch 2/750
4000/4000 [==============================] - 1s 228us/sample - loss: 46.8191 - mae: 46.8191 - mse: 17945.0977
Epoch 3/750
4000/4000 [==============================] - 1s 207us/sample - loss: 46.4166 - mae: 46.4166 - mse: 17749.5039
Epoch 4/750
4000/4000 [==============================] - 1s 202us/sample - loss: 46.4766 - mae: 46.4766 - mse: 17748.6113
Epoch 5/750
4000/4000 [==============================] - 1s 239us/sample - loss: 46.1579 - mae: 46.1578 - mse: 17709.7656
Epoch 6/750
4000/4000 [==============================] - 1s 202us/sample - loss: 46.1312 - mae: 46.1312 - mse: 17657.6543
Epoch 7/750
4000/4000 [==============================] - 1s 220us/sample - loss: 46.0582 - mae: 46.0582 - mse: 17689.8730s - loss: 46.6309 - mae: 46.6309 - mse:
Epoch 8/750
4000/4000 [===================

4000/4000 [==============================] - 1s 238us/sample - loss: 45.3259 - mae: 45.3259 - mse: 17553.1016
Epoch 67/750
4000/4000 [==============================] - 1s 220us/sample - loss: 45.3927 - mae: 45.3927 - mse: 17550.5801
Epoch 68/750
4000/4000 [==============================] - 1s 247us/sample - loss: 45.4090 - mae: 45.4090 - mse: 17568.9805
Epoch 69/750
4000/4000 [==============================] - 1s 226us/sample - loss: 45.3232 - mae: 45.3232 - mse: 17487.8398
Epoch 70/750
4000/4000 [==============================] - 1s 213us/sample - loss: 45.2512 - mae: 45.2512 - mse: 17517.8047
Epoch 71/750
4000/4000 [==============================] - 1s 211us/sample - loss: 45.3904 - mae: 45.3904 - mse: 17533.4160
Epoch 72/750
4000/4000 [==============================] - 1s 218us/sample - loss: 45.3079 - mae: 45.3079 - mse: 17534.7266
Epoch 73/750
4000/4000 [==============================] - 1s 208us/sample - loss: 45.1954 - mae: 45.1954 - mse: 17499.3789
Epoch 74/750
4000/4000 [=====

4000/4000 [==============================] - 1s 261us/sample - loss: 44.5726 - mae: 44.5727 - mse: 17318.8477
Epoch 133/750
4000/4000 [==============================] - 1s 300us/sample - loss: 44.5780 - mae: 44.5780 - mse: 17285.7188
Epoch 134/750
4000/4000 [==============================] - 1s 282us/sample - loss: 44.5865 - mae: 44.5865 - mse: 17305.7441
Epoch 135/750
4000/4000 [==============================] - 1s 322us/sample - loss: 44.6679 - mae: 44.6680 - mse: 17390.4688
Epoch 136/750
4000/4000 [==============================] - 1s 282us/sample - loss: 44.5716 - mae: 44.5716 - mse: 17261.2012
Epoch 137/750
4000/4000 [==============================] - 1s 274us/sample - loss: 44.5309 - mae: 44.5309 - mse: 17328.8574
Epoch 138/750
4000/4000 [==============================] - 1s 286us/sample - loss: 44.6392 - mae: 44.6392 - mse: 17267.1152
Epoch 139/750
4000/4000 [==============================] - 1s 364us/sample - loss: 44.6548 - mae: 44.6548 - mse: 17320.4844
Epoch 140/750
4000/400

4000/4000 [==============================] - 1s 307us/sample - loss: 44.1454 - mae: 44.1454 - mse: 17185.9727
Epoch 199/750
4000/4000 [==============================] - 2s 511us/sample - loss: 44.1250 - mae: 44.1250 - mse: 17197.7383
Epoch 200/750
4000/4000 [==============================] - 1s 335us/sample - loss: 44.2421 - mae: 44.2421 - mse: 17274.4902
Epoch 201/750
4000/4000 [==============================] - 1s 332us/sample - loss: 44.1152 - mae: 44.1152 - mse: 17151.6445
Epoch 202/750
4000/4000 [==============================] - 1s 351us/sample - loss: 43.9966 - mae: 43.9966 - mse: 17158.6191
Epoch 203/750
4000/4000 [==============================] - 1s 318us/sample - loss: 44.0959 - mae: 44.0959 - mse: 17214.9609
Epoch 204/750
4000/4000 [==============================] - 1s 258us/sample - loss: 43.9706 - mae: 43.9706 - mse: 17205.5840
Epoch 205/750
4000/4000 [==============================] - 1s 307us/sample - loss: 44.0684 - mae: 44.0684 - mse: 17134.2949
Epoch 206/750
4000/400

4000/4000 [==============================] - 1s 356us/sample - loss: 43.6871 - mae: 43.6871 - mse: 17035.7754
Epoch 265/750
4000/4000 [==============================] - 1s 313us/sample - loss: 43.7835 - mae: 43.7835 - mse: 16999.8574
Epoch 266/750
4000/4000 [==============================] - 1s 236us/sample - loss: 43.8243 - mae: 43.8243 - mse: 17050.6074
Epoch 267/750
4000/4000 [==============================] - 1s 369us/sample - loss: 43.7192 - mae: 43.7192 - mse: 17052.7754
Epoch 268/750
4000/4000 [==============================] - 2s 451us/sample - loss: 43.5920 - mae: 43.5920 - mse: 17070.1895
Epoch 269/750
4000/4000 [==============================] - 1s 315us/sample - loss: 43.6436 - mae: 43.6436 - mse: 16993.7832
Epoch 270/750
4000/4000 [==============================] - 1s 277us/sample - loss: 43.5900 - mae: 43.5900 - mse: 17057.4883
Epoch 271/750
4000/4000 [==============================] - 1s 281us/sample - loss: 43.5546 - mae: 43.5547 - mse: 17035.8086
Epoch 272/750
4000/400

...

4000/4000 [==============================] - 1s 294us/sample - loss: 43.7803 - mae: 43.7803 - mse: 16476.2363
Epoch 464/750
4000/4000 [==============================] - 1s 341us/sample - loss: 43.8340 - mae: 43.8340 - mse: 16589.8984
Epoch 465/750
4000/4000 [==============================] - 1s 332us/sample - loss: 43.8055 - mae: 43.8055 - mse: 16468.1270
Epoch 466/750
4000/4000 [==============================] - 1s 292us/sample - loss: 43.8220 - mae: 43.8220 - mse: 16500.6504
Epoch 467/750
4000/4000 [==============================] - 1s 306us/sample - loss: 43.6704 - mae: 43.6704 - mse: 16464.9648
Epoch 468/750
4000/4000 [==============================] - 1s 374us/sample - loss: 43.7409 - mae: 43.7409 - mse: 16511.7285
Epoch 469/750
4000/4000 [==============================] - 2s 408us/sample - loss: 43.8017 - mae: 43.8017 - mse: 16453.9883
Epoch 470/750
4000/4000 [==============================] - 1s 274us/sample - loss: 43.7853 - mae: 43.7853 - mse: 16484.1758
Epoch 471/750
4000/400

4000/4000 [==============================] - 1s 282us/sample - loss: 43.3055 - mae: 43.3055 - mse: 16281.1279
Epoch 530/750
4000/4000 [==============================] - 1s 317us/sample - loss: 43.1722 - mae: 43.1722 - mse: 16216.7217
Epoch 531/750
4000/4000 [==============================] - 1s 351us/sample - loss: 43.1815 - mae: 43.1814 - mse: 16245.7158
Epoch 532/750
4000/4000 [==============================] - 1s 324us/sample - loss: 43.1983 - mae: 43.1982 - mse: 16176.9697
Epoch 533/750
4000/4000 [==============================] - 1s 328us/sample - loss: 43.1386 - mae: 43.1386 - mse: 16158.3164
Epoch 534/750
4000/4000 [==============================] - 1s 317us/sample - loss: 43.0698 - mae: 43.0698 - mse: 16272.7656
Epoch 535/750
4000/4000 [==============================] - 1s 283us/sample - loss: 43.0846 - mae: 43.0846 - mse: 16196.7441
Epoch 536/750
4000/4000 [==============================] - 1s 271us/sample - loss: 43.0626 - mae: 43.0626 - mse: 16233.6094
Epoch 537/750
4000/400

4000/4000 [==============================] - 1s 312us/sample - loss: 42.7148 - mae: 42.7148 - mse: 15995.5596
Epoch 596/750
4000/4000 [==============================] - 1s 285us/sample - loss: 42.7207 - mae: 42.7208 - mse: 16062.8838
Epoch 597/750
4000/4000 [==============================] - 1s 281us/sample - loss: 42.6875 - mae: 42.6875 - mse: 15939.0928
Epoch 598/750
4000/4000 [==============================] - 1s 274us/sample - loss: 42.5872 - mae: 42.5872 - mse: 16018.0088
Epoch 599/750
4000/4000 [==============================] - 1s 291us/sample - loss: 42.7575 - mae: 42.7575 - mse: 15998.9648
Epoch 600/750
4000/4000 [==============================] - 1s 314us/sample - loss: 42.7601 - mae: 42.7601 - mse: 15948.6357
Epoch 601/750
4000/4000 [==============================] - 1s 282us/sample - loss: 42.5737 - mae: 42.5738 - mse: 15967.3652
Epoch 602/750
4000/4000 [==============================] - 1s 278us/sample - loss: 42.6560 - mae: 42.6560 - mse: 15957.5059
Epoch 603/750
4000/400

4000/4000 [==============================] - 1s 312us/sample - loss: 42.2237 - mae: 42.2238 - mse: 15690.4365
Epoch 662/750
4000/4000 [==============================] - 1s 282us/sample - loss: 42.1833 - mae: 42.1833 - mse: 15672.1084
Epoch 663/750
4000/4000 [==============================] - 1s 282us/sample - loss: 42.1848 - mae: 42.1848 - mse: 15741.7734
Epoch 664/750
4000/4000 [==============================] - 1s 275us/sample - loss: 42.3592 - mae: 42.3592 - mse: 15856.5615
Epoch 665/750
4000/4000 [==============================] - 1s 293us/sample - loss: 42.2017 - mae: 42.2017 - mse: 15724.1104
Epoch 666/750
4000/4000 [==============================] - 1s 310us/sample - loss: 42.2090 - mae: 42.2090 - mse: 15652.2441
Epoch 667/750
4000/4000 [==============================] - 1s 278us/sample - loss: 42.4057 - mae: 42.4057 - mse: 15761.5166
Epoch 668/750
4000/4000 [==============================] - 1s 279us/sample - loss: 42.1602 - mae: 42.1602 - mse: 15733.2217
Epoch 669/750
4000/400

4000/4000 [==============================] - 1s 314us/sample - loss: 41.9390 - mae: 41.9390 - mse: 15544.7002
Epoch 727/750
4000/4000 [==============================] - 1s 277us/sample - loss: 41.9588 - mae: 41.9588 - mse: 15600.2539
Epoch 728/750
4000/4000 [==============================] - 1s 275us/sample - loss: 41.8218 - mae: 41.8218 - mse: 15577.3564
Epoch 729/750
4000/4000 [==============================] - 1s 275us/sample - loss: 41.8049 - mae: 41.8049 - mse: 15510.2969
Epoch 730/750
4000/4000 [==============================] - 1s 281us/sample - loss: 41.6083 - mae: 41.6083 - mse: 15459.4121
Epoch 731/750
4000/4000 [==============================] - 1s 326us/sample - loss: 41.9676 - mae: 41.9676 - mse: 15578.5371
Epoch 732/750
4000/4000 [==============================] - 1s 280us/sample - loss: 41.7024 - mae: 41.7024 - mse: 15509.8740
Epoch 733/750
4000/4000 [==============================] - 1s 283us/sample - loss: 41.8832 - mae: 41.8832 - mse: 15444.0137
Epoch 734/750
4000/400

In [13]:
print(scores)
print("Mean MAE:", scores.mean())
print("Standard Deviation of the mean:", scores.std())

[-46.9850619  -39.2342574  -41.49781449 -37.39725642 -40.22649268]
Mean MAE: -41.06817657985687
Standard Deviation of the mean: 3.247813627445587
